# 경쟁적 전염: 백준 18405번

1,2,3 순으로 확장을 하고나서, 
N초 뒤에 (x,y)를 찾으면 되겠다.
BFS로 문제를 풀이하면 될 것 같다.

In [39]:
#N by N형태의 graph, 1번부터 K번까지의 바이러스
#결과는 시간초과다. 
N,K = map(int, input().split())
graph = []
birus = [[] for _ in range(K+1)] # birus의 좌표를 담아내야할듯
for i in range(N):
    graph.append(list(map(int, input().split())))
    for j in range(N):
        if graph[i][j] == 0:
            continue
        else:
            birus[graph[i][j]].append((0,i,j,graph[i][j])) #시간, x,y,value

s,a,b = map(int, input().split())

# 1 --> 2 --> 3 순으로 q에 넣으면 될듯?
from collections import deque
q = deque(birus)

dx = [1,-1,0,0] #하 상 우 좌 
dy = [0,0,1,-1]

flag = False
while q: #바이러스를 하나씩 증식시킨다.
    bottle = q.popleft() #birus가 담겨있는 병이 있다고 생각하자.
    empty = [] # 빈 병
    for i in range(len(bottle)):
        t,x,y,value = bottle[i]
        if t == s:
            flag = True
            break
        for j in range(4):
            tx = x + dx[j]
            ty = y + dy[j]
            if tx < 0 or tx >= N or ty < 0 or ty >= N:
                continue
            if graph[tx][ty] == 0:
                graph[tx][ty] = value
                empty.append((t+1,tx,ty,value))
    if flag:
        break
    else:
        q.append(empty)

print(graph[a-1][b-1])

3 3
1 0 2
0 0 0
3 0 0
0 2 2


In [54]:
# 우선순위 큐를 사용하면 될듯?
import heapq
q = []

N,K = map(int, input().split())
graph = []
for i in range(N):
    graph.append(list(map(int,input().split())))
    for j in range(N):
        if graph[i][j] != 0:
            heapq.heappush(q,(0,graph[i][j],i,j))

s,a,b = map(int,input().split())

dx = [1,-1,0,0]
dy = [0,0,1,-1]

while q:
    time, value, x, y = heapq.heappop(q)
    if time == s:
        break
    for i in range(4):
        tx = x + dx[i]
        ty = y + dy[i]
        if tx < 0 or tx >= N or ty < 0 or ty >= N:
            continue
        if graph[tx][ty] == 0:
            graph[tx][ty] = value
            heapq.heappush(q,(time+1,value,tx,ty))
print(graph[a-1][b-1])

3 3
1 0 2
0 0 0
3 0 0
1 2 2
0


그냥 q로 담는 것도 좋은 생각이지만, 그런 경우 while문 안에 한 번의 반복문이 더 추가되서 결과적으로 3개의 iteration이 돌아가는 시간초과가 발생한다.
이를 heapq로 사용하면 해결할 수 있다. --> 아니면 sort를 한번 잡아주면 되는 구나.

참고로 heapq(우선순위큐)를 사용해서, 여러개의 값을 통째로 넣으면 (x1,x2,x3,x4) x1별로 오름차순 정렬, x2별로 오름차순 정렬을 거쳐서 만들어준다. 


In [ ]:
from collections import deque

n, k = map(int, input().split())

graph = [] # 전체 보드 정보를 담는 리스트
data = [] # 바이러스에 대한 정보를 담는 리스트

for i in range(n):
    # 보드 정보를 한 줄 단위로 입력
    graph.append(list(map(int, input().split())))
    for j in range(n):
        # 해당 위치에 바이러스가 존재하는 경우
        if graph[i][j] != 0:
            # (바이러스 종류, 시간, 위치 X, 위치 Y) 삽입
            data.append((graph[i][j], 0, i, j))

# 정렬 이후에 큐로 옮기기 (낮은 번호의 바이러스가 먼저 증식하므로)
data.sort()
q = deque(data)
 
target_s, target_x, target_y = map(int, input().split())
 
# 바이러스가 퍼져나갈 수 있는 4가지의 위치
dx = [-1, 0, 1, 0]
dy = [0, 1, 0, -1]

# 너비 우선 탐색(BFS) 진행
while q:
    virus, s, x, y = q.popleft()
    # 정확히 s초가 지나거나, 큐가 빌 때까지 반복
    if s == target_s:
        break
    # 현재 노드에서 주변 4가지 위치를 각각 확인
    for i in range(4):
        nx = x + dx[i]
        ny = y + dy[i]
        # 해당 위치로 이동할 수 있는 경우
        if 0 <= nx and nx < n and 0 <= ny and ny < n:
            # 아직 방문하지 않은 위치라면, 그 위치에 바이러스 넣기
            if graph[nx][ny] == 0:
                graph[nx][ny] = virus
                q.append((virus, s + 1, nx, ny))

print(graph[target_x - 1][target_y - 1])

# 아기상어: 백준16236번

조건:

1) 아기상어의 크기는 2
<br>
2) 상하좌우로 이동가능
<br>
3) 자기보다 작은 물고기 먹을 수 있음 , 같은 물고기는 지나가기만 가능함
<br>
4) 자신의 크기와 동일한 개수의 물고기를 먹어야 size가 하나 증가한다. --> 경험치 같은 느낌이랄까

5) 최단거리 <br>
가장 위쪽 --> 가장 왼쪽

6) 먹을 수 있는지 없는지를 판단해야 하는데, 


'''
갑자기 막힌 이유는, 최단거리에 위치한 애를 잡으러 가는 과정에서
자기보다 큰 상어들이 존재한다면, 불가능하다..
이런 경우를 생각해보지 못한 것 같다.

--> 뭔가 BFS를 재귀함수로 돌려보면 어떨까 생각해본다.
'''

# BFS를 통해 발견할때까지 search를 하면 되지 않을까?

In [155]:
# 매순간마다 최단거리에 있는 target을 계산해주어야 하는데, 일단해보자
from collections import deque
N = int(input())
graph2 = []
for i in range(N):
    graph2.append(list(map(int, input().split())))
    if 9 in graph2[i]:
        start_x, start_y = i, graph2[i].index(9)

6
5 4 3 2 3 4
4 3 2 3 4 5
3 2 9 5 6 6
2 1 2 3 4 5
3 2 1 6 5 4
6 6 6 6 6 6


In [141]:
5 4 3 2 3 4
4 3 2 3 4 5
3 2 9 5 6 6
2 1 2 3 4 5
3 2 1 6 5 4
6 6 6 6 6 6

[[4, 3, 2, 1], [0, 0, 0, 0], [0, 0, 9, 0], [1, 2, 3, 4]]

In [177]:
import copy
graph = copy.deepcopy(graph2)
# queue를 활용해서, 9 주위로 값을 넣어보자
# 상, 좌, 우, 하
dx = [-1,0,0,1]
dy = [0,-1,1,0]

# visited를 사용해야 무한루프에서 벗어날 수 있다.

# 먹을 때까지 돌리자.
def bfs(row,col,level):
    visited = [[False]*N for _ in range(N)]
    q = deque()
    q.append((row,col,0))
    while q:
        x,y,walk = q.popleft()
        for i in range(4):
            tx = x + dx[i]
            ty = y + dy[i]
            #영역을 벗어낫거나 level이 더 높다면,
            if tx < 0 or tx >= N or ty < 0 or ty >= N or graph[tx][ty] > level:
                continue
            # 0 이 아니 &고 자기보다 작다면 사냥 가능
            elif graph[tx][ty] != 0 and graph[tx][ty] < level:
                return (tx,ty,walk+1)
            elif not visited[tx][ty]: #아직 방문하지 않은 곳이라면
                visited[tx][ty] = True # 방문체크함.
                q.append((tx,ty,walk+1))
    return (-1,-1,-1)

result = 0
a,b,cnt = (start_x,start_y,0)
level = 2
prey = 0

token = 0
while True:
    token += 1
    graph[a][b] = 0
    a,b,cnt = bfs(a,b,level) #row,col,이동거리
    print("row = {}, col = {}, 이동거리 = {}, level = {}".format(a+1,b+1,cnt,level))
    if token %5 == 0:
        print()

    
    #더이상 먹이를 찾지 못했다면.
    if (a,b,cnt) == (-1,-1,-1):
        break
    
    # 아기상어의 위치가 바뀌지.
    graph[a][b] = 9
    
    #이동거리(시간)을 result에 넣어줌
    result += cnt
    #먹이cnt + 1
    prey += 1
    # level만큼의 먹이를 먹었다면, level을 증가시키고, prey 는 0으로 바꿔줌
    if level == prey:
        level += 1
        prey = 0

print(result)

row = 1, col = 2, 이동거리 = 9, level = 2
row = 1, col = 1, 이동거리 = 1, level = 2
row = 2, col = 1, 이동거리 = 1, level = 3
row = 2, col = 2, 이동거리 = 1, level = 3
row = 3, col = 2, 이동거리 = 1, level = 3

row = 3, col = 1, 이동거리 = 1, level = 4
row = 4, col = 1, 이동거리 = 1, level = 4
row = 4, col = 2, 이동거리 = 1, level = 4
row = 4, col = 3, 이동거리 = 1, level = 4
row = 4, col = 4, 이동거리 = 1, level = 5

row = 3, col = 4, 이동거리 = 1, level = 5
row = 2, col = 4, 이동거리 = 1, level = 5
row = 1, col = 4, 이동거리 = 1, level = 5
row = 1, col = 3, 이동거리 = 1, level = 5
row = 1, col = 5, 이동거리 = 2, level = 6

row = 1, col = 6, 이동거리 = 1, level = 6
row = 2, col = 6, 이동거리 = 1, level = 6
row = 2, col = 5, 이동거리 = 1, level = 6
row = 3, col = 5, 이동거리 = 1, level = 6
row = 3, col = 6, 이동거리 = 1, level = 6

row = 4, col = 6, 이동거리 = 1, level = 7
row = 4, col = 5, 이동거리 = 1, level = 7
row = 5, col = 6, 이동거리 = 2, level = 7
row = 3, col = 3, 이동거리 = 5, level = 7
row = 2, col = 3, 이동거리 = 1, level = 7

row = 0, col = 0, 이동거리 = -1, level = 7
39


In [ ]:
6
5 4 3 2 3 4
4 3 2 3 4 5
3 2 9 5 6 6
2 1 2 3 4 5
3 2 1 6 5 4
6 6 6 6 6 6
정답:60

6
1 1 1 1 1 1
2 2 6 2 2 3
2 2 5 2 2 3
2 2 2 4 6 3
0 0 0 0 0 6
0 0 0 0 0 9
정답:39
    
6
6 0 6 0 6 1
0 0 0 0 0 2
2 3 4 5 6 6
0 0 0 0 0 2
0 2 0 0 0 0
3 9 3 0 0 1
정답:48


In [183]:
#수정본
import copy
graph = copy.deepcopy(graph2)
# queue를 활용해서, 9 주위로 값을 넣어보자
# 상, 좌, 우, 하
dx = [-1,0,0,1]
dy = [0,-1,1,0]

# visited를 사용해야 무한루프에서 벗어날 수 있다.

# 먹을 때까지 돌리자.
def bfs(row,col,level):
    visited = [[False]*N for _ in range(N)]
    q = deque()
    q.append((row,col,0))
    ans = (-1,-1,-1)
    previous = 0
    flag = False # previous 가 바뀌었음을 알려주기
    while q:
        if flag == True and previous != ans[-1]:
            return ans
        
        x,y,walk = q.popleft()
        # 이전 거리보다 walk였을 때 거리가 더 커졌다면

        for i in range(4):
            tx = x + dx[i]
            ty = y + dy[i]
            #영역을 벗어낫거나 level이 더 높다면,
            if tx < 0 or tx >= N or ty < 0 or ty >= N or graph[tx][ty] > level:
                continue
            # 0 이 아니 &고 자기보다 작다면 사냥 가능
            elif graph[tx][ty] != 0 and graph[tx][ty] < level:
                flag = True
                if ans == (-1,-1,-1):
                    ans = (tx,ty,walk+1)
                elif tx < ans[0] and walk+1 <= ans[2]: # 더 위쪽에 있다
                    ans = (tx,ty,walk+1)
                elif tx == ans[0] and ty < ans[1] and walk+1 <= ans[2]:
                    ans = (tx,ty,walk+1)
                     
            elif not visited[tx][ty]: #아직 방문하지 않은 곳이라면
                visited[tx][ty] = True # 방문체크함.
                q.append((tx,ty,walk+1))
                previous = walk + 1
        #가까운 거리 중에서 확인
    return ans

result = 0
a,b,cnt = (start_x,start_y,0)
level = 2
prey = 0

token = 0
while True:
    token += 1
    graph[a][b] = 0
    a,b,cnt = bfs(a,b,level) #row,col,이동거리
    #더이상 먹이를 찾지 못했다면.
    if (a,b,cnt) == (-1,-1,-1):
        break
    # 아기상어의 위치가 바뀌지.
    graph[a][b] = 9
    
    #이동거리(시간)을 result에 넣어줌
    result += cnt
    #먹이cnt + 1
    prey += 1
    # level만큼의 먹이를 먹었다면, level을 증가시키고, prey 는 0으로 바꿔줌
    if level == prey:
        level += 1
        prey = 0

print(result)

48


# 정리:
1) BFS를 통해서 기본적인 문제를 풀이하였다.

2) visited를 통해서, 해당 부분을 방문했는지 확인하였다.

3) 같은 거리의 최단거리에 놓여있다면, 위 --> 왼부터 search를 진행하였는데, 이 부분이 상당히 난황이였다. 

--> 코드 자체가 정리가 잘 안되어있지만, 해결해서 뿌듯하다


In [187]:
#나동빈 code
from collections import deque
INF = 1e9

n = int(input())

    
# 아기 상어의 현재 크기와 위치변수
now_size = 2
now_x, now_y = 0,0

array = []
for i in range(n):
    array.append(list(map(int, input().split())))
    for j in range(n):
        if array[i][j] == 9:
            now_x, now_y = i,j
            array[now_x][now_y] = 0

# 상, 우, 하, 좌
dx = [-1,0,1,0]
dy = [0,1,0,-1]

# 최단거리만 계산하는 BFS
def bfs():
    # -1이라면 도달할 수 없음을 의미한다.
    dist = [[-1]*n for _ in range(n)]
    
    q = deque([(now_x, now_y)])
    dist[now_x][now_y] = 0
    while q:
        x,y = q.popleft()
        for i in range(4):
            nx = x + dx[i]
            ny = y + dy[i]
            if 0 <= nx < n and 0 <= ny < n:
                if dist[nx][ny] == -1 and array[nx][ny] <= now_size:
                    dist[nx][ny] = dist[x][y] + 1
                    q.append((nx,ny))
    return dist

# 최단거리 테이블이 주어졌을 때, 먹이 찾기
def find(dist):
    x, y = 0,0
    min_dist =INF
    for i in range(n):
        for j in range(n):
            #위에서 부터 시작했을때 현재 now_size보다 작은 경우를 훑는다.
            if dist[i][j] != -1 and 1 <= array[i][j] < now_size:
                # 훝었을 때 distance가 더 작은 경우
                if dist[i][j] < min_dist:
                    x,y = i,j
                    min_dist = dist[i][j]
    if min_dist == INF:
        return None
    else:
        return x,y,min_dist
    
result = 0
ate = 0

while True:
    value = find(bfs())
    if value == None:
        print(result)
        break
    else:
        now_x,now_y = value[0],value[1]
        result += value[2]
        
        array[now_x][now_y] = 0
        ate += 1
        if ate >= now_size:
            now_size += 1
            ate = 0        

6
5 4 3 2 3 4
4 3 2 3 4 5
3 2 9 5 6 6
2 1 2 3 4 5
3 2 1 6 5 4
6 6 6 6 6 6
60


In [ ]:
6
5 4 3 2 3 4
4 3 2 3 4 5
3 2 9 5 6 6
2 1 2 3 4 5
3 2 1 6 5 4
6 6 6 6 6 6
정답:60